In [ ]:
import os
import re
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.optim as optim
from tqdm import tqdm

print("=== 프로그램이 시작되었습니다 ===")

# 1. 커스텀 데이터셋 클래스
class DeepfakeDataset(Dataset):
    def __init__(self, root_dir):
        self.image_paths = []
        self.labels = []

        for label, category in enumerate(['real', 'fake']):
            category_path = os.path.join(root_dir, category)
            for gender in ['남성', '여성']:
                gender_path = os.path.join(category_path, gender)
                if not os.path.exists(gender_path):
                    continue
                for filename in os.listdir(gender_path):
                    if filename.lower().endswith(('.jpg', '.png')):
                        full_path = os.path.join(gender_path, filename)
                        self.image_paths.append(full_path)
                        self.labels.append(label)

        self.to_tensor = transforms.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            image = Image.open(self.image_paths[idx]).convert('RGB')
        except Exception:
            image = Image.new('RGB', (64, 64), (0, 0, 0))

        image = self.to_tensor(image)
        label = self.labels[idx]
        return image, label

# 2. 데이터셋 및 DataLoader 설정
train_dir = 'archive/frames/train'
val_dir = 'archive/frames/validate'

train_dataset = DeepfakeDataset(train_dir)
val_dataset = DeepfakeDataset(val_dir)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# 3. 모델, 장치, 손실함수, 옵티마이저 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. 저장된 마지막 에폭 모델 불러오기
checkpoint_dir = '.'
latest_epoch = 0

for filename in os.listdir(checkpoint_dir):
    match = re.match(r'resnet18_epoch(\d+)\.pth', filename)
    if match:
        epoch_num = int(match.group(1))
        if epoch_num > latest_epoch:
            latest_epoch = epoch_num

if latest_epoch > 0:
    checkpoint_path = f'resnet18_epoch{latest_epoch}.pth'
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"🔄 이전 학습 모델 로드 완료 (epoch {latest_epoch})")
else:
    print("🆕 새로 학습을 시작합니다")

# 최고 정확도 저장용 변수
best_val_acc = 0.0

# 5. 학습 루프
num_epochs = 10
print("===== 학습을 시작합니다 =====")

for epoch in range(latest_epoch, num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", ncols=100)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=100 * correct / total)

    train_acc = correct / total
    print(f"Epoch {epoch+1} 완료 - Loss: {total_loss:.4f}, Accuracy: {train_acc:.4f}")

    # 모델 저장
    save_path = f'resnet18_epoch{epoch+1}.pth'
    torch.save(model.state_dict(), save_path)
    print(f"💾 에폭 {epoch+1} 모델 저장 완료: {save_path}")

    # 검증
    print("🔍 검증 중...")
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total if val_total > 0 else 0
    print(f"📊 Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'resnet18_best.pth')
        print(f"🌟 새로운 최고 정확도 모델 저장됨 (acc: {val_acc:.4f})")

print("✅ 전체 학습 완료")


=== 프로그램이 시작되었습니다 ===


C:\Users\singa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\singa\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🔄 이전 학습 모델 로드 완료 (epoch 8)
===== 학습을 시작합니다 =====


Epoch 9/10:  43%|███████▎         | 166307/386688 [3:00:11<4:13:23, 14.50it/s, acc=99.4, loss=0.175]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 9/10:  68%|█████████▌    | 264752/386688 [4:59:07<2:23:56, 14.12it/s, acc=99.4, loss=0.000131]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 9/10:  80%|████████████▋   | 308000/386688 [6:07:16<1:42:10, 12.84it/s, acc=99.4, loss=0.0265]IOPub message rate exceeded.
The Jupyter server will temporarily

Epoch 9 완료 - Loss: 6284.7327, Accuracy: 0.9941
💾 에폭 9 모델 저장 완료: resnet18_epoch9.pth
🔍 검증 중...
📊 Validation Accuracy: 0.7272
🌟 새로운 최고 정확도 모델 저장됨 (acc: 0.7272)


Epoch 10/10:   0%|                   | 830/386688 [00:49<5:58:49, 17.92it/s, acc=99.4, loss=6.97e-5]